In [28]:
import pandas as pd
import numpy as np
from io import StringIO
Train = pd.read_csv("Data/train.csv")
Test = pd.read_csv("Data/test.csv")

In [29]:
# fill null values with median for temp_apache
Train['temp_apache'].fillna(Train['temp_apache'].median(), inplace=True)

# fill null values with median for d1_potassium_max
Train['d1_potassium_max'].fillna(Train['d1_potassium_max'].median(), inplace=True)

# fill null values with median for apache_4a_hospital_death_prob
Train['apache_4a_hospital_death_prob'].fillna(Train['apache_4a_hospital_death_prob'].median(), inplace=True)

# fill null values with median for apache_4a_icu_death_prob
Train['apache_4a_icu_death_prob'].fillna(Train['apache_4a_icu_death_prob'].median(), inplace=True)
#since all these r heavily skewed andaffected by the outliers we will fill them using median imputation

Test['temp_apache'].fillna(Test['temp_apache'].median(), inplace=True)

# fill null values with median for d1_potassium_max
Test['d1_potassium_max'].fillna(Test['d1_potassium_max'].median(), inplace=True)

# fill null values with median for apache_4a_hospital_death_prob
Test['apache_4a_hospital_death_prob'].fillna(Test['apache_4a_hospital_death_prob'].median(), inplace=True)

# fill null values with median for apache_4a_icu_death_prob
Test['apache_4a_icu_death_prob'].fillna(Test['apache_4a_icu_death_prob'].median(), inplace=True)

# group the dataframe by apache_2_bodysystem and calculate the mean age for each group
mean_age_by_bodysystem = Train.groupby('apache_2_bodysystem')['age'].mean()

# define a function that takes a row of the dataframe as input and returns the mean age of the corresponding apache_2_bodysystem
def fill_age(row):
    if pd.isnull(row['age']):
        if pd.isnull(row['apache_2_bodysystem']):
            return np.nan
        else:
            return mean_age_by_bodysystem[row['apache_2_bodysystem']]
    else:
        return row['age']

# apply the function to each row of the dataframe and fill the missing age values with the corresponding mean age
Train['age'] = Train.apply(fill_age, axis=1)

# group the dataframe by apache_2_bodysystem and calculate the mean age for each group
mean_age_by_bodysystem = Test.groupby('apache_2_bodysystem')['age'].mean()

# define a function that takes a row of the dataframe as input and returns the mean age of the corresponding apache_2_bodysystem
def fill_age(row):
    if pd.isnull(row['age']):
        if pd.isnull(row['apache_2_bodysystem']):
            return np.nan
        else:
            return mean_age_by_bodysystem[row['apache_2_bodysystem']]
    else:
        return row['age']

# apply the function to each row of the dataframe and fill the missing age values with the corresponding mean age
Test['age'] = Test.apply(fill_age, axis=1)

#for all binary columns we will apply mode imputation for missing values
#first we will create a list of all binary columns
binary_colsTest = ['elective_surgery', 'apache_post_operative', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache','immunosuppression', 'solid_tumor_with_metastasis']

binary_colsTrain = ['elective_surgery', 'apache_post_operative', 'gcs_unable_apache', 'intubated_apache', 'ventilated_apache','immunosuppression', 'solid_tumor_with_metastasis','hospital_death']
#now we will apply mode imputation on these columns
from sklearn.impute import SimpleImputer
binary_colsTest = [col for col in Train.columns if Train[col].dtype == 'object' or col in binary_colsTest]
binary_colsTrain = [col for col in Test.columns if Test[col].dtype == 'object' or col in binary_colsTrain]

imputer = SimpleImputer(strategy='most_frequent')
Train[binary_colsTrain] = imputer.fit_transform(Train[binary_colsTrain])
Test[binary_colsTest] = imputer.fit_transform(Test[binary_colsTest])

numeric_cols = [col for col in Train.select_dtypes(include=[np.number]).columns if col not in binary_colsTrain]
numeric_colsTest = [col for col in Test.select_dtypes(include=[np.number]).columns if col not in binary_colsTrain]

from sklearn.impute import KNNImputer

# create an instance of KNNImputer with k=3
imputer = KNNImputer(n_neighbors=3)

# fill missing values in Train dataframe
Train[numeric_cols] = imputer.fit_transform(Train[numeric_cols])

# fill missing values in Test dataframe
Test[numeric_colsTest] = imputer.fit_transform(Test[numeric_colsTest])

dropcolumns=['d1_diasbp_noninvasive_min','h1_sysbp_max','h1_mbp_max', 'h1_mbp_noninvasive_max', 'h1_sysbp_noninvasive_max', 'd1_sysbp_noninvasive_min', 'h1_diasbp_noninvasive_min']
Train= Train.drop(dropcolumns, axis=1)
Test= Test.drop(dropcolumns, axis=1)

#next we drop simialr record columns
Train = Train.drop('apache_3j_bodysystem', axis=1)
Test = Test.drop('apache_3j_bodysystem', axis=1)

In [17]:
#one hot encode Train and Test
Traindum = pd.get_dummies(Train)
Testdum = pd.get_dummies(Test)


In [18]:
#scale using zscore standerdization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Traindum = scaler.fit_transform(Traindum)
Testdum = scaler.fit_transform(Testdum)



In [27]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
Train_scaled = scaler.fit_transform(Train)
Train_scaled = pd.DataFrame(Train_scaled, columns=Train.columns)


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [6]:

import time
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from catboost import CatBoostClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split 
from sklearn.metrics import roc_curve, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [20]:
import pandas as pd

In [13]:
X = Train.loc[: , Train.columns != 'hospital_death']
y = Train['hospital_death']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

KeyError: 'hospital_death'

In [ ]:
#record the start time
nb_c = CatBoostClassifier(iterations=650, depth=3, learning_rate=0.1, loss_function='Logloss', verbose=False)
nb_c.fit(X_train,y_train)
md_probs = nb_c.predict_proba(X_val)
md_probs = md_probs[:,1]
md_auc = roc_auc_score(y_val, md_probs)
print("Cat Boost" , " : ", md_auc)